<a href="https://colab.research.google.com/github/trongthuan205/DS102.K21/blob/master/02_CT_Covid19_Detection_Using_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 . Import library

In [0]:
import cv2
import glob
import imageio 
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [0]:
link_Covid_train = "/content/drive/My Drive/Datasets/CT_Covid19/CT_Covid_Train/*"
link_nonCovid_train = "/content/drive/My Drive/Datasets/CT_Covid19/CT_Non_Covid_Train/*"
link_Covid_test =  "/content/drive/My Drive/Datasets/CT_Covid19/CT_Covid_Test/*"
link_nonCovid_test = "/content/drive/My Drive/Datasets/CT_Covid19/CT_Non_Covid_Test/*"

# 1 . Load data

In [0]:
X_train = []
X_test = []
y_train = []
y_test = []

In [0]:
def load_data(path, X, y, result):
    for im_path in glob.glob(path):
        cv_img = cv2.imread(im_path);     
        gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)   
        img = cv2.resize(gray,(28,28))   
        X.append(img)
        y.append(result)

In [0]:
load_data(link_Covid_train, X_train, y_train, 1)

In [0]:
load_data(link_nonCovid_train, X_train, y_train, 0)

In [0]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [0]:
X_train.shape

In [0]:
y_train.shape

In [0]:
load_data(link_Covid_test, X_test, y_test, 1)

In [0]:
load_data(link_nonCovid_test, X_test, y_test, 0)

In [0]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [0]:
X_test.shape

In [0]:
y_test.shape

# 2 . Statistic data

In [0]:
Counter(y_train)

In [0]:
Counter(y_test)

# 3 . Visualize data

In [0]:
#Visulize random image
img = X_train[0,:]
plt.imshow(img)

In [0]:
#Visualize top 10 positive in train data
index = np.where(y_train == 1)[0][:10]
plt.figure(figsize = (15,10))
for i, j in enumerate(index):
    plt.subplot(3, 5, i+1)
    plt.imshow(X_train[j])

In [0]:
#Visualize top 10 negative in train data
index = np.where(y_train == 0)[0][:10]
plt.figure(figsize = (15,10))
for i, j in enumerate(index):
    plt.subplot(3, 5, i+1)
    plt.imshow(X_train[j])

# 4 . Normalize data

In [0]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_train.shape

In [0]:
y_train  = y_train.reshape(-1,1)
y_train.shape

In [0]:
X_test = X_test.reshape(X_test.shape[0], -1)
X_test.shape

In [0]:
y_test = y_test.reshape(-1, 1)
y_test.shape

# 5 . Build model

In [0]:
model = KNeighborsClassifier(n_neighbors = 9)
model.fit(X_train, y_train)

In [0]:
y_train_pred = model.predict(X_train)

In [0]:
y_test_pred = model.predict(X_test)

# 6 . Evaluate

Train data

In [0]:
print('Accuracy score: ', accuracy_score(y_train, y_train_pred))
print('Confusion matrix: \n', confusion_matrix(y_train, y_train_pred))
plot_confusion_matrix(model, X_train, y_train)

Test data

In [0]:
print('Accuracy score: ', accuracy_score(y_test, y_test_pred))
print('Confusion matrix: \n', confusion_matrix(y_test, y_test_pred))
plot_confusion_matrix(model, X_test, y_test)

# 7 . Visualize result

In [0]:
def VisualizingDataset(X_, y_):
    X1 = X_[:,0]
    X2 = X_[:,1]
    for i, label in enumerate(np.unique(y_)):
        plt.scatter(X1[y_ == label], X2[y_ == label], color = ListedColormap(('red', 'green'))(i))
        plt.legend(np.unique(y_))

In [0]:
def VisualizingResult(model, X_):
    X1 = X_[:,0]
    X2 = X_[:,1]
    X1_range = np.arange(X1.min()-1, X1.max()+1, 0.01)
    X2_range = np.arange(X2.min()-1, X2.max()+1, 0.01)
    X1_matrix, X2_matrix = np.meshgrid(X1_range, X2_range)
    X_grid = np.array([X1_matrix.ravel(), X2_matrix.ravel()]).T
    y_grid = model.predict(X_grid).reshape(X1_matrix.shape)
    plt.contourf(X1_matrix, X2_matrix, y_grid, alpha = 0.5, cmap = ListedColormap(('red','green')))

In [0]:
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
#VisualizingDataset(X_train, y_train)
VisualizingResult(model, X_train)
plt.title('Train data')
plt.subplot(1,2,2)
#VisualizingDataset(X_test, y_test)
VisualizingResult(model, X_test)
plt.title('Test data')
plt.show()